<a href="https://colab.research.google.com/github/nupursjsu/Deep-Learning/blob/master/Ungraded_Assignment3/Building_own_DNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building our own DNN model

## Importing necessary libraries

In [0]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
#Setting default size for plots
plt.rcParams['figure.figsize'] = (5.0, 4.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

#Setting seed to have consistent results
np.random.seed(12)

## Getting testCases_v4a and dnn_utils_v2 packages to import from

In [0]:
# Code to read files into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
#Getting testCases_v4a package file
link = 'https://drive.google.com/open?id=1rCNvN1f2Q8Xcz6vU6QPTq9xnhuVIB1Ct'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testCases_v4a.py')

1rCNvN1f2Q8Xcz6vU6QPTq9xnhuVIB1Ct


In [5]:
#Getting dnn_utils_v2 package file
link = 'https://drive.google.com/open?id=1GGim5lLwKn_nPDeszE-JQefEom72jiLC'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dnn_utils_v2.py')

1GGim5lLwKn_nPDeszE-JQefEom72jiLC


In [0]:
#Importing functions from the packages
from testCases_v4a import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

## Initializing model parameters

In [0]:
def initialize_params(x, h, ny):
    
    np.random.seed(12)
    
    W1 = np.random.randn(h,x)*0.01
    b1 = np.zeros((h,1))
    W2 = np.random.randn(y, h)*0.01
    b2 = np.zeros((y,1))

    assert(W1.shape == (h, x))
    assert(b1.shape == (h, 1))
    assert(W2.shape == (y, h))
    assert(b2.shape == (y, 1))
    
    params = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return params

## Initializing L-layer Neural Network parameters

In [0]:
def initialize_params_deep(layer_dims):
  
    np.random.seed(3)
    params = {}
    L = len(layer_dims)          

    for l in range(1, L):
        params['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        params['b' + str(l)] = np.zeros((layer_dims[l], 1))
        assert(params['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(params['b' + str(l)].shape == (layer_dims[l], 1))

        
    return params

## Implementing Forward propagation module

In [0]:
#Building the linear part of forward propagation module
def linear_fwd(A, W, b):
    
    Z = np.dot(W,A)+b

    assert(Z.shape == (W.shape[0], A.shape[1]))
    op = (A, W, b)
    
    return Z, op

In [0]:
#Implementing the forward propagation of the LINEAR->ACTIVATION layer
def linear_activation_fwd(A_prev, W, b, activation):
    
    if activation == "sigmoid":
     
        Z, linear_op = linear_fwd(A_prev, W, b)
        A, activation_op = sigmoid(Z)
       
    elif activation == "relu":
     
        Z, linear_op = linear_fwd(A_prev, W, b)
        A, activation_op = relu(Z)

    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    op = (linear_op, activation_op)

    return A, op

In [0]:
#Implementing the forward propagation of the L-layer model
def L_model_fwd(X, params):

    ops = []
    A = X
    L = len(params) // 2                
    for l in range(1, L):
        A_prev = A 
        A, op = linear_activation_fwd(A_prev, 
                                             params['W' + str(l)], 
                                             params['b' + str(l)], 
                                             activation='relu')
        ops.append(op)
        
    AL, op = linear_activation_fwd(A, 
                                          params['W' + str(L)], 
                                          params['b' + str(L)], 
                                          activation='sigmoid')
    ops.append(op)
    assert(AL.shape == (1,X.shape[1]))
    return AL, ops

## Implementing loss function

In [0]:
def compute_loss(AL, Y):

    r = Y.shape[1]

    loss = (-1/r)* np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1-Y, np.log(1-AL)))

    loss = np.squeeze(loss)    
    assert(loss.shape == ())
    
    return loss

## Implementing Backward propagation module

In [0]:
#Implementing linear backward
def linear_bwd(dZ, op):
    A_prev, W, b = op
    r = A_prev.shape[1]

    dW = (1/r)*np.dot(dZ, A_prev.T)
    db = (1/r)*np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T,dZ)
  
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [0]:
#Implementing the backward propagation of the LINEAR->ACTIVATION layer
def linear_activation_bwd(dA, op, activation):
   
    linear_op, activation_op = op
    
    if activation == "relu":
       
        dZ = relu_backward(dA, op[1])
        dA_prev, dW, db = linear_bwd(dZ, linear_op)
      
    elif activation == "sigmoid":
       
        dZ = sigmoid_backward(dA, op[1])
        dA_prev, dW, db = linear_bwd(dZ, linear_op)
  
    return dA_prev, dW, db

In [0]:
#Implementing the backward propagation of the L-layer model
def L_model_bwd(AL, Y, ops):

    gradients = {}
    
    L = len(ops) 
    r = AL.shape[1]
    Y = Y.reshape(AL.shape) 
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    current_op = ops[L-1]
    gradients["dA" + str(L-1)], gradients["dW" + str(L)], gradients["db" + str(L)] = linear_activation_bwd(dAL, current_op, activation = "sigmoid")
    for l in reversed(range(L-1)):

        current_op = ops[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_bwd(gradients["dA" + str(l + 1)], current_op, activation = "relu")
        gradients["dA" + str(l)] = dA_prev_temp
        gradients["dW" + str(l + 1)] = dW_temp
        gradients["db" + str(l + 1)] = db_temp
    return gradients

## Updating parameters using gradient descent

In [0]:
## Updating parameters using gradient descent
def update_params(params, gradients, learning_rate):
    
    L = len(params) // 2 

    for l in range(L):
        params["W" + str(l+1)] = params["W" + str(l+1)] - learning_rate * gradients["dW" + str(l + 1)]
        params["b" + str(l+1)] = params["b" + str(l+1)] - learning_rate * gradients["db" + str(l + 1)]

    return params

## Conclusion

In this assignment I learned how to build our own deep neural network model